<a href="https://colab.research.google.com/github/jmquintana/data_science_sprint_2/blob/main/JoseQuintanaProyecto_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sprint Proyect 2**
**Ingeniería de Features, Modelos Avanzados e Interpretación de Modelos (4 sprints)**

## **Story Points**
<img src='https://s3.amazonaws.com/platform-resources.acamica.com/slides/Im%C3%A1genes+en+Toolboxes/DS+(4+Sprints)/ds_sprintproject2_a_actuallizado.png' >

### **Transformación de Datos**
***

#### **Carga del dataset**

In [ ]:
# Importamos las librerías que vamos a necesitar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from google.colab import data_table

In [735]:
# Le doy formato a los gráficos
sns.set_style("darkgrid")
plt.rc('font', size=10)
plt.rc('axes', titlesize=16)
plt.rc('figure', titlesize=16)
plt.rc('axes', labelsize=14) 
plt.rc('xtick', labelsize=12) 
plt.rc('ytick', labelsize=12) 

In [736]:
WORK_WITH_SAMPLE = False # esto al inicio de todo

In [737]:
#Seteamos para que no utilice notacion cientifica
pd.options.display.float_format = '{:.4f}'.format
#Seteo para que el máximo de columnas que muestra al levantar una base sean 500
pd.set_option('display.max_columns',500)
#Estos códigos hacen que la visualización de la consola abarque toda la pantalla (sin los recortes a los costados). Tambien hacen que al mostrar dataframes podamos ver todas las columnas que tiene.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# Codigo para poder imprimir multiples outputs en una misma línea
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
%load_ext google.colab.data_table

In [738]:
# Monto mi Google Drive para cargar el DataSet
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [739]:
# Cargo el DataSet con Pandas como un DataFrame nombrado "df"
# Previamente debe descargarse del siguiente link: https://drive.google.com/uc?export=download&id=1Ugbsw5XbNRbglomSQO1qkAgMFB_3BzmB
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DS_Proyecto_01_Datos_Properati.csv")
# df = pd.read_csv("DS_Proyecto_01_Datos_Properati.csv")

KeyboardInterrupt: ignored

In [ ]:
if WORK_WITH_SAMPLE:
  df = df.sample(n=10000, random_state=99)

In [ ]:
print("Columnas:",df.shape[1])
print("Filas:",df.shape[0])

In [ ]:
df.head()

In [ ]:
# Elimino columnas que no necesitaré y renombro las columnas
df = df[['lat', 'lon', 'l2', 'l3', 'property_type', 'rooms', 'bedrooms', 'bathrooms', 'surface_total', 'surface_covered', 'price']]
df = df.rename({'l2':'zona', 'l3':'barrio', 'rooms':'ambientes', 'bedrooms':'dormitorios', 'bathrooms':'baños', 'surface_total':'sup_total', 'surface_covered':'sup_cubierta', 'price':'precio', 'property_type':'tipo'}, axis=1)
%load_ext google.colab.data_table
data_table.DataTable(df, include_index=False, num_rows_per_page=10)

In [ ]:
# Filtro por las propiedades de la zona de Capital Federal
%unload_ext google.colab.data_table
df = df[df.zona == 'Capital Federal']
df.head()

In [ ]:
# Agrupo las propiedades por `tipo` para ver su participación dentro de la muestra
df_group = df.groupby('tipo').count().precio / df.shape[0]
df_group.sort_values(ascending=False, inplace=True)
df_group = df_group.reset_index(name='rel')
# df_group['rel'] = pd.Series(["{0:.1f}%".format(val * 100) for val in df_group['rel']])
df_group

In [ ]:
# Descarto las instancias del dataset cuyo ´tipo´ tenga una articipación menor al 1% del total (ya que no considero representativa)
# Me quedo con los tipos Departamento, Casa y PH
df = df[df.tipo.isin(['Departamento', 'Casa', 'PH'])]

A continuación realizamos algunas verificaciones extra: que no haya `sup_cubierta` > `sup_total`, que no haya instancias duplicadas.

In [ ]:
# Verifico que la superfice cubierta no sea mayor que la total
# A continuación vamos a filtrar aquellas propiedades que posean 'surface_covered' > 'surface_total' ya que son  inconsistencias del dataset.
mascara = (df.sup_cubierta) > (df.sup_total)
print("La cantidad de instancias que tienen 'sup_cubierta' mayor que 'sup_total' es:", df[mascara].shape[0])

In [ ]:
# Reasignamos esas instancias con el valor de `sup_total`
df.loc[mascara, 'sup_cubierta'] = df.loc[mascara, 'sup_total']

In [ ]:
# Verificamos las instancias duplicadas
df.drop_duplicates(inplace=True)

In [ ]:
df.shape

In [ ]:
def hmap(df):
  plt.figure(figsize=(12,10))
  corr_matrix = df.corr()
  mask = np.zeros_like(corr_matrix)
  mask[np.triu_indices_from(mask)] = True
  sns.heatmap(corr_matrix, vmin=-0.5, annot=True, square=True, mask=mask, cmap='BrBG')
  plt.show()
  
hmap(df)

#### **Elección de transformaciones**

##### 1. Encoding

Vamos a realizar el encoding de las variables categóricas: ***zona***, ***barrio*** y ***tipo*** para poder utilizar éstos atributos como input del modelo predictor. Las 3 variables son de tipo nominal por lo que aplicaría ***One Hot Enconding***. Sin embargo, la columna ***barrio*** presenta muchas instancias distintas, por lo que realizar un One Hot Encoding de ella generaría muchas columnas adicionales y encarecería el procesamiento de los datos.

##### 2. Análisis de Valores Faltantes

In [ ]:
# Cantidad de valores faltantes por columna del dataset
faltantes = pd.DataFrame(df.isnull().sum(), columns=['faltantes'])
faltantes

###### 2.1. Faltantes de variable `baños`

In [ ]:
# Análisis faltantes en variable 'baños'
# Agrupo los Baños faltantes por Tipo de propiedad
df_baños_faltantes = df.baños.isnull().groupby([df['tipo']]).sum().astype(int).reset_index(name='missings')
df_baños_faltantes['total'] = df.precio.groupby(df['tipo']).count().values
df_baños_faltantes['rel'] = df_baños_faltantes.missings / df_baños_faltantes.total
df_baños_faltantes.sort_values(by='rel', ascending=False, inplace=True)
df_baños_faltantes['%'] = pd.Series(["{0:.2f}%".format(val * 100) for val in df_baños_faltantes['rel']])
df_baños_faltantes

In [ ]:
# Análisis faltantes en variable 'baños'
faltantes_baño = df[df.baños.isna()]
tipos = faltantes_baño.tipo.value_counts()
plt.figure(figsize=(6,5))
plt.title('Cantidad porcentual de faltantes de Baño por Tipo de propiedad')
ax = sns.barplot(data=df_baños_faltantes, x='tipo', y='rel')
plt.ylabel('Cantidad de faltantes (%)')
plt.xlabel('Tipo de propiedad')
plt.xticks(rotation=0)
ax.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1))
plt.show()

Para solucionar el tema de los valores faltantes de la variable `baños` podría imputarlos con el valor de la moda para cada `tipo` de propiedad.

In [ ]:
sns.boxplot(data=df, y='tipo', x='baños', order=df_baños_faltantes.tipo)
plt.show()

In [ ]:
# La moda de la variable `baños` por cada tipo de propiedad
moda_baños_por_tipo = df.groupby(['tipo']).agg(lambda x:x.value_counts().index[0]).baños.reset_index(name='moda_baños')
moda_baños_por_tipo

###### 2.2. Faltantes de variable `sup_total` y `sup_cubierta`

In [ ]:
# Separo los datos que tienen missings de superficie (total o cubierta)
mask = np.logical_or(df.sup_cubierta.isna(), df.sup_total.isna())
faltantes_sup = df[mask]

def sup_faltante(row):
  if np.logical_and(pd.isna(row.sup_total), pd.isna(row.sup_cubierta)):
    return 'ambas'
  elif np.logical_and(pd.isna(row.sup_total), ~pd.isna(row.sup_cubierta)):
    return 'total'
  elif np.logical_and(~pd.isna(row.sup_total), pd.isna(row.sup_cubierta)):
    return 'cubierta'
  else:
    return 'ninguna'

faltantes_sup['falta sup'] = faltantes_sup.apply(sup_faltante, axis=1)
faltantes_sup.head()

In [ ]:
# Vemos las instancias en las que faltan datos de superficie cubierta y/o de superficie total
faltantes_sup.groupby('falta sup').count().precio.reset_index(name='missings')

In [ ]:
sns.countplot(data=faltantes_sup, x='falta sup')
plt.show()

Para los casos en que sólo falta una de las superficies, imputaría el valor de la faltante con el valor de la otra.
Para el caso en que faltan ambas superficies las imputaría calculando el precio por metro cuadrado promedio de cada barrio y luego calculo la superficie total dividiendo el `precio` por el `precio_m2` calculado por `barrio`.

###### 2.3. Faltantes de variables `lat` y `lon`

In [ ]:
# Separo los datos que tienen missings de superficie (total o cubierta)
mask = np.logical_or(df.lat.isna(), df.lon.isna())
faltantes_coord = df[mask]

def coord_faltante(row):
  if np.logical_and(pd.isna(row.lat), pd.isna(row.lon)):
    return 'ambas'
  elif np.logical_and(pd.isna(row.lat), ~pd.isna(row.lon)):
    return 'lat'
  elif np.logical_and(~pd.isna(row.lat), pd.isna(row.lon)):
    return 'lon'
  else:
    return 'ninguna'

faltantes_coord['falta coordenada'] = faltantes_coord.apply(coord_faltante, axis=1)
faltantes_coord.head()

In [ ]:
faltantes_coord.groupby('falta coordenada').count().precio.reset_index(name='missings')

In [ ]:
sns.countplot(data=faltantes_coord, x='falta coordenada')
plt.show()

En este caso completaría las coordenadas `lat` y `lon` de modo de imputarles el promedio de los valores correspondientes a los `barrios` a los cuales pertenecen las instancias.

In [ ]:
df_con_coord = df[~mask]
barrios = df_con_coord.groupby(by='barrio').mean()[['lat', 'lon']].reset_index()
barrios

##### 3. Detección y Eliminación de Outliers

#### **Implementación de transformaciones**

##### 1. Missings

In [ ]:
# Creamos un diccionario para imputar los missings
dict_moda_baños_por_tipo = moda_baños_por_tipo.set_index('tipo').transpose().to_dict('records')[0]
print(dict_moda_baños_por_tipo)

In [ ]:
# Imputando missings en `baños`
df.baños = df.baños.fillna(df.tipo.map(dict_moda_baños_por_tipo))
df

In [ ]:
# Verificamos que no haya más missings en `baños`
df.isna().sum()

In [ ]:
# Imputando missings en `lat` y `lon`
dict_media_lat_por_barrio = barrios[['barrio','lat']].set_index('barrio').transpose().to_dict('records')[0]
dict_media_lon_por_barrio = barrios[['barrio','lon']].set_index('barrio').transpose().to_dict('records')[0]
df.lat = df.lat.fillna(df.barrio.map(dict_media_lat_por_barrio))
df.lon = df.lon.fillna(df.barrio.map(dict_media_lon_por_barrio))
df.isna().sum()

In [ ]:
# Agregamos una columna auxiliar al dataframe para identificar facilmente si falta
# una superficie o las dos, para facilitar la imputación de missings de superficie.
df['falta_sup'] = df.apply(sup_faltante, axis=1)
df.head()

In [ ]:
df.falta_sup.unique()

In [ ]:
# Imputamos los missings de `sup_total` con los valores de `sup_cubierta`
df.loc[df[df.falta_sup == 'total'].index, 'sup_total'] = df[df.falta_sup == 'total'].sup_cubierta

In [ ]:
# Imputamos los missings de `sup_cubierta` con los valores de `sup_total`
df.loc[df[df.falta_sup == 'cubierta'].index, 'sup_cubierta'] = df[df.falta_sup == 'cubierta'].sup_total

In [ ]:
# Verificamos que ahora tenemos misma cantidad de missings de `sup_cubierta` y `sup_total`
df.isna().sum()

In [ ]:
# Creamos una columna calculada con el precio por metro cuadrado
df['precio_m2'] = df.precio / df.sup_total
# Y luego creamos un diccionario con el precio por metro cuadrado por barrio
dict_precio_m2_por_barrio = df.groupby('barrio').mean()['precio_m2'].to_dict()

In [ ]:
# Luego asigno el precio por metro cuadrado de cada barrio a cada fila que contiene faltantes de precio por m2 (debido al faltante de superficie)
df.precio_m2 = df.precio_m2.fillna(df.barrio.map(dict_precio_m2_por_barrio))
# E imputo los valores faltantes de superficie con el siguiente cálculo: precio dividido el precio por metro cuadrado.
df.sup_total.fillna(df.precio / df.precio_m2, inplace=True)
# Finalmente imputo la superficie cubierta con el valor de superficie total.
df.sup_cubierta.fillna(df.sup_total, inplace=True)

In [ ]:
# Verifico que no tengo más missings.
df.isna().sum()

In [ ]:
# Descarto las columnas auxiliares que no usaremos más.
df.drop(columns=['precio_m2', 'falta_sup'], inplace=True)

##### 2. Encoding

In [ ]:
! pip install feature_engine

In [ ]:
pip install --upgrade category_encoders

In [ ]:
# Importamos las librerias que son utiles para esto
import feature_engine
from feature_engine.imputation import AddMissingIndicator, CategoricalImputer, MeanMedianImputer, ArbitraryNumberImputer
from feature_engine.encoding import RareLabelEncoder
from feature_engine.outliers import Winsorizer
from feature_engine.selection import DropConstantFeatures, DropCorrelatedFeatures, SmartCorrelatedSelection
import category_encoders as ce
from imblearn.pipeline import Pipeline

Posible opción para encoding: asignar un número a cada categoría:

Asignar un entero a cada valor de las variables categóricas (tiene la desventaja de que los valores asignados no guardan relación alguna entre ellas, por ejemplo, no se puede inferir por medio de los números asignados si un `barrio` es en promedio más caro que otro, si está más cerca o si las superficies de sus propiedades guardan alguna relación).

In [ ]:
from sklearn.preprocessing import LabelEncoder
df2 = df.copy()
cols = ('barrio', 'tipo')
# process columns, apply LabelEncoder to categorical features
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(df2[c].values)) 
    df2[c] = lbl.transform(list(df[c].values))

# shape        
print('Shape all_data: {}'.format(df2.shape))

In [ ]:
df2.head()

Otra opción es realizar un ***Target Encoding***.

Este tipo de encoding permite 'ayudar' al modelo, dar alguna pauta de la relación que existe entre los valores de variables categóricas.
Por ejemplo, si hacemos encoding de los `barrios` con relación a la variable `precio`, se podría inferir que a valores más altos de `barrio_code` se corresponden, por lo general, valores más altos de `precio`. 

In [ ]:
df3 = df.copy() # Partimos del dataset orginal
cols = ('zona', 'barrio', 'tipo')

for col in cols:
    lbl=ce.TargetEncoder(cols=col)
    df3[col + '_code'] = round(lbl.fit_transform(df3[col], df3['precio']), 0)

df3.shape

In [ ]:
df3.head()

In [ ]:
# Graficamos el heatmap para monitorear las correlaciones
hmap(df3)

In [ ]:
df4 = df3.copy()

###### 1.1. One Hot Encoder

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

In [ ]:
data = df[['barrio', 'tipo', 'sup_total', 'precio']]
column_trans = make_column_transformer(
    (OneHotEncoder(sparse=False),['barrio', 'tipo']),
    remainder='passthrough'
)
data = data.dropna()
X = data[['barrio', 'tipo', 'sup_total']]
y = data['precio']
print(X.shape)
print(y.shape)

In [ ]:
X.head()

In [ ]:
column_trans.fit_transform(X)

##### 3. Ouliers

In [ ]:
%unload_ext google.colab.data_table
df3.describe()

La superficie mínima (tanto la total como la cubierta) siguen siendo de 1 $m^2$ (valor que no tiene sentido), por lo que voy a filtrar el dataset para superficies mayores a 25 $m^2$.

In [ ]:
df3 = df3[df3.sup_total > 25]
df3 = df3[df3.sup_cubierta > 25]
df3.shape

In [ ]:
df3.describe()

In [ ]:
# Vamos a eliminar los outliers con el método del rango intercuartílico.
df_out = df3.copy()
def removeOutliers(data, col):
    Q3 = np.quantile(data[col], 0.75)
    Q1 = np.quantile(data[col], 0.25)
    IQR = Q3 - Q1
      
    print("El IQR de la columna %s es: %s" % (col, IQR))
    global outlier_free_list
    global filtered_data

    lower_range = Q1 - 1.5 * IQR
    upper_range = Q3 + 1.5 * IQR
    outlier_free_list = [x for x in data[col] if (
        (x > lower_range) & (x < upper_range))]
    filtered_data = data.loc[data[col].isin(outlier_free_list)]

columns_to_filter = ['sup_total', 'sup_cubierta', 'precio', 'ambientes', 'dormitorios', 'baños']

for i in columns_to_filter:
    removeOutliers(df_out, i)
    # Assigning filtered data back to our orginal variable
    df_out = filtered_data
print("Shape of data after outlier removal is: ", df_out.shape)

In [ ]:
def dist_plot(df_in, df_out, col):
  plt.figure(figsize=(11,1))
  sns.boxplot(data=df_in, x=col)
  plt.title('Boxplot del Dataset original')
  plt.show()
  plt.figure(figsize=(11,1))
  sns.boxplot(data=df_out, x=col)
  plt.title('Boxplot del Dataset sin outliers')
  plt.show()
  plt.figure(figsize=(10,4))
  g = sns.histplot(data=df_out, x=col, kde=True, bins=50, hue='tipo')
  plt.title("Histograma del Dataset sin outliers")
  plt.ylabel("Cantidad de Propiedades")
  plt.show()

dist_plot(df, df_out, 'sup_total')

In [ ]:
cols = ['sup_total', 'sup_cubierta', 'precio', 'ambientes', 'dormitorios', 'baños']
dist_plot(df, df_out, cols[1])

In [ ]:
dist_plot(df, df_out, cols[2])

In [ ]:
dist_plot(df, df_out, cols[3])

In [ ]:
dist_plot(df, df_out, cols[4])

In [ ]:
dist_plot(df, df_out, cols[5])

In [ ]:
df_out.shape

In [ ]:
# Cantidad de instancias por Tipo de Propiedad
tipos = df_out['tipo'].value_counts().to_frame("Cantidad")
tipos

In [ ]:
# Generamos un gráfico de la distribución de la variable Precio por Tipo de propiedad y las Cantidades de Propiedades por Tipo
fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharey=True, gridspec_kw={'width_ratios': [3, 1]})
fig.suptitle('Distribución de Precios y Cantidad de Propiedades por Tipo')

# Boxplot
sns.boxplot(ax=axes[0], x=df_out.precio, y=df_out.tipo, order=tipos.index)
axes[0].set_xlabel('Precio [usd]')
axes[0].set_ylabel('Tipo de propiedad')

# Barplot
ax = sns.barplot(ax=axes[1], x=tipos.Cantidad, y=tipos.index, order=tipos.index)
axes[1].set_xlabel('Cantidad')
axes[1].set_ylabel('')

# label each bar in barplot
for p in ax.patches:
  height = p.get_height() # height of each horizontal bar is the same
  width = max(tipos.values)/2
  # adding text to each bar
  ax.text(x = width, # x-coordinate position of data label, padded 3 to right of bar
    y = p.get_y()+(height/2), # # y-coordinate position of data label, padded to be in the middle of the bar
    s = '{:.0f}'.format(p.get_width()),
    va = 'center',
    fontdict= { 'fontsize': 12})

plt.show()

In [ ]:
df_out.describe()

In [ ]:
df_out.shape

#### **Reentrenamiento de <u>Modelo Sprint 1<u>**

##### Reentrenamiento de modelo

In [ ]:
# Definimos las variables de predictoras y la variable a predecir
data = df_out.copy()
X = data.drop(['zona', 'barrio', 'tipo', 'precio'], axis=1)
y = data['precio']

In [ ]:
# En caso que la variable predictora sea una sola, graficamos la variable predecir en función de la predictora.
# Si no se cumple la condición, no se realiza el gráfico.
if X.shape[1] == 1:
  plt.figure(figsize=(10, 8))
  plt.scatter(X,y, s = 2)
  plt.xlabel('Superficie total [m2]')
  plt.ylabel('Precio [usd]')
  plt.legend()
  plt.show()

In [ ]:
# Hacemos la división entre datos de entrenamiento y datos de testeo.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
# Importamos las librerías de los modelos a utilizar
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

# Instanciamos lo modelos
linear_model = LinearRegression()
tree_regressor = DecisionTreeRegressor(max_depth=11, random_state=0, max_features=2, min_samples_leaf=0.00015704594766823636, min_samples_split= 0.0003043067752731219, criterion='mae')
knn_regressor = KNeighborsRegressor(n_neighbors=3)

In [ ]:
# Entrenamos los modelos
linear_model.fit(X_train, y_train)
tree_regressor.fit(X_train, y_train)
knn_regressor.fit(X_train, y_train)

In [ ]:
# En caso que la variable predictora sea una sola, graficamos la variable predecir en función de la predictora para los tres modelos.
if X.shape[1] == 1 :
  plt.figure(figsize = (20,10))

  plt.subplot(1,3,1)
  plt.scatter(X,y, s = 2)
  plt.plot(X,linear_model.predict(X),label ='Regresion Lineal', c = 'g')
  plt.xlabel('x')
  plt.ylabel('y')
  plt.legend()

  plt.subplot(1,3,2)
  plt.scatter(X,y, s = 2)
  plt.plot(X,tree_regressor.predict(X),label ='Árbol de Decisión', c = 'g')

  plt.xlabel('x')
  plt.ylabel('y')
  plt.legend()

  plt.subplot(1,3,3)
  plt.scatter(X,y, s = 2)
  plt.plot(X,knn_regressor.predict(X),label ='knn', c = 'g')

  plt.xlabel('x')
  plt.ylabel('y')
  plt.legend()

  plt.tight_layout()
  plt.show()

In [ ]:
# Luego analizamos la distribución de los errores de las predicciones de cada modelo
# y calculamos la raíz del error cuadrático medio de la cada modelo con los hiperparámetros utilizados
from sklearn.metrics import mean_squared_error
import seaborn as sns
modelos = ['Regresión lineal', 'Árbol de Decisión', 'Vecinos más cercanos']

for i, model in enumerate([linear_model, tree_regressor, knn_regressor]):
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    print(f'Modelo: {modelos[i]}')

    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    print(f'Raíz del error cuadrático medio en Train: {rmse_train}')
    print(f'Raíz del error cuadrático medio en Test: {rmse_test}')
    
    plt.figure(figsize = (16,4))

    plt.subplot(1,2,1)
    sns.histplot(y_train - y_train_pred, bins = 20, label = 'train', kde=True)
    sns.histplot(y_test - y_test_pred, bins = 20, label = 'test', kde=True)
    plt.xlabel('errores')
    plt.xlim(-200000, 200000)
    plt.ylim(0, 50000)
    plt.legend()


    ax = plt.subplot(1,2,2)
    ax.scatter(y_test,y_test_pred, s =2)
    
    lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes]
    ]
    
    ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    plt.xlabel('y (test)')
    plt.ylabel('y_pred (test)')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Ahora vamos a analizar el R2 que puede darnos una idea más comparable del 
# ajuste de los datos predichos con los datos reales de la muestra.
# Importamos la metrica
from sklearn.metrics import r2_score

In [ ]:
# Primero calculamos el R^2 del benchmark: Modelo de Regresión Lineal
r2_test_lin = r2_score(y_test,linear_model.predict(X_test))
r2_train_lin = r2_score(y_train,linear_model.predict(X_train))
print('r2_train_lin:', r2_train_lin)
print('r2_test_lin:', r2_test_lin)

In [ ]:
# Vamos a calcular los R2 variando los hiperparámetros de entrenamiento del modelo
# Definimos las listas vacias para los valores de accuracy deseados
lista_rsme_train = []
lista_rsme_test = []
lista_r2_train = []
lista_r2_test = []

# Definimos la lista de valores de k que vamos a explorar
max_depths = np.arange(1,25, 1)

# Generamos un loop sobre los distintos valores de k 
for d in max_depths:
    
    # Vamos a repetir el siguiente bloque de código
    
    # Definir el modelo con el valor de vecinos deseado
    clf = DecisionTreeRegressor(max_depth = d)
    
    # Entrenar el modelo
    clf.fit(X_train, y_train)
    

    # Predecir y evaluar sobre el set de entrenamiento
    y_train_pred = clf.predict(X_train)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    r2_train = r2_score(y_train, y_train_pred)
    # train_acc = clf.score(X_train, y_train_pred)
    
    # Predecir y evaluar sobre el set de evaluación
    y_test_pred = clf.predict(X_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    r2_test = r2_score(y_test, y_test_pred)
    # test_acc = clf.score(X_test, y_test_pred)
    
    # Agregar la información a las listas
    lista_rsme_train.append(rmse_train)
    lista_rsme_test.append(rmse_test)
    lista_r2_train.append(r2_train)
    lista_r2_test.append(r2_test)
  

In [ ]:
# A continuación creamos el gráfico de los R^2 en función de los niveles de profundidad máxima del modelo de Árboles de decisión.
# Adicionalmente, marcamos el punto óptimo que maximiza el R2 en los datos de testeo.
# Este punto sería el máximo a considerar para definir la profundidad del Árbol de decisión, ya que queremos minimizar el costo computacional de entrenamiento.
fig, ax1 = plt.subplots(figsize=(12,6))

x = max_depths
y1 = lista_r2_train
y2 = lista_r2_test
y3 = lista_rsme_train
y4 = lista_rsme_test

ax2 = ax1.twinx()
ax1.plot(x, y1, 'o-', label='r2 train') # r2 train
ax1.plot(x, y2, 'o-', label='r2 test') # r2 test
ax2.plot(x, y3, '--', label='rsme train', alpha=1) # rsme train
ax2.plot(x, y4, '--', label='rsme test', alpha=1) # rsme test

ax1.set_xlabel('Niveles de profundidad')
ax1.set_ylabel('$R^2$', rotation=0, size=14, labelpad=20)
ax2.set_ylabel('$RMSE$', rotation=0, size=14, labelpad=30)

max_r2_test_tree = max(y2)
max_depth = x[np.where(y2 == max_r2_test_tree)[0][0]]
r_train_tree = y1[np.where(y2 == max_r2_test_tree)[0][0]]

ax1.plot(max_depth, max_r2_test_tree, '*r', markersize=20)
ax1.plot(max_depth, r_train_tree, '*r', markersize=20)
y_min = ax1.get_ylim()[0]
y_max = ax1.get_ylim()[1]
plt.axvline(x=max_depth, color='r', ymax=(r_train_tree - y_min) / (y_max-y_min), dashes=(4,4))

ax1.annotate(np.round(max_r2_test_tree,2), xy=(max_depth, max_r2_test_tree), xytext=(20, -25), size=14, xycoords='data', textcoords='offset points')
ax1.annotate(np.round(r_train_tree, 2), xy=(max_depth, r_train_tree), xytext=(20, -20), size=14,xycoords='data', textcoords='offset points')
ax1.annotate("max_depth = " + str(np.round(max_depth, 0)), xy=(max_depth, y_min), xytext=(-140, 20), size=14,xycoords='data', textcoords='offset points')

plt.title('Ajuste Árboles de decisión')
ax1.legend(fontsize=12, loc='best', bbox_to_anchor=(0.5, 0.4, 0.5, 0.5))
ax2.legend(fontsize=12, loc='best', bbox_to_anchor=(0.5, -0.2, 0.5, 0.5))

plt.show()

In [ ]:
# Repetimos lo anterior, pero para el modelo de Vecinos más cercanos
# Definimos las listas vacias para los valores de accuracy deseados
lista_rsme_train = []
lista_rsme_test = []
lista_r2_train = []
lista_r2_test = []
# time_list = []
# fisrt_time = milisecs(datetime.now())

# Definimos la lista de valores de k que vamos a explorar
k_vecinos = np.arange(1,27, 1)

# Generamos un loop sobre los distintos valores de k 
for k in k_vecinos:
    
    # Vamos a repetir el siguiente bloque de código
    
    # Definir el modelo con el valor de vecinos deseado
    clf = KNeighborsRegressor(n_neighbors = k)
    
    # Entrenar el modelo
    clf.fit(X_train, y_train)

    # Predecir y evaluar sobre el set de entrenamiento
    y_train_pred = clf.predict(X_train)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    r2_train = r2_score(y_train, y_train_pred)
    # train_acc = clf.score(X_train, y_train_pred)
    # time = milisecs(datetime.now()) - fisrt_time
    
    # Predecir y evaluar sobre el set de evaluación
    y_test_pred = clf.predict(X_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    r2_test = r2_score(y_test, y_test_pred)
    # test_acc = clf.score(X_test, y_test_pred)
    
    # Agregar la información a las listas
    lista_rsme_train.append(rmse_train)
    lista_rsme_test.append(rmse_test)
    lista_r2_train.append(r2_train)
    lista_r2_test.append(r2_test)
    # time_list.append(time)

In [ ]:
# A continuación creamos el gráfico de los R^2 en función de la cantidad de vecinos del modelo de KNN.
fig, ax1 = plt.subplots(figsize=(12,6))

x = k_vecinos
y1 = lista_r2_train
y2 = lista_r2_test
y3 = lista_rsme_train
y4 = lista_rsme_test

ax2 = ax1.twinx()
ax1.plot(x, y1, 'o-', label='r2 train') # r2 train
ax1.plot(x, y2, 'o-', label='r2 test') # r2 test
ax2.plot(x, y3, '--', label='rsme train', alpha=1) # rsme train
ax2.plot(x, y4, '--', label='rsme test', alpha=1) # rsme test

ax1.set_xlabel('K vecinos más cercanos')
ax1.set_ylabel('$R^2$', rotation=0, size=14, labelpad=20)
ax2.set_ylabel('$RMSE$', rotation=0, size=14, labelpad=30)

max_r2_test_tree = max(y2)
max_depth = x[np.where(y2 == max_r2_test_tree)[0][0]]
r_train_tree = y1[np.where(y2 == max_r2_test_tree)[0][0]]

ax1.plot(max_depth, max_r2_test_tree, '*r', markersize=20)
ax1.plot(max_depth, r_train_tree, '*r', markersize=20)
y_min = ax1.get_ylim()[0]
y_max = ax1.get_ylim()[1]
plt.axvline(x=max_depth, color='r', ymax=(r_train_tree - y_min) / (y_max-y_min), dashes=(4,4))

ax1.annotate(np.round(max_r2_test_tree,2), xy=(max_depth, max_r2_test_tree), xytext=(20, -25), size=14, xycoords='data', textcoords='offset points')
ax1.annotate(np.round(r_train_tree, 2), xy=(max_depth, r_train_tree), xytext=(20, -20), size=14,xycoords='data', textcoords='offset points')
ax1.annotate("knn = " + str(np.round(max_depth, 0)), xy=(max_depth, y_min), xytext=(-80, 20), size=14,xycoords='data', textcoords='offset points')

plt.title('Ajuste K vecinos más cercanos')
ax1.legend(fontsize=12, loc='lower right')
ax2.legend(fontsize=12, loc='upper right')

plt.show()

##### Evaluación de desempeño

In [ ]:
# Creamos una tabla con los resultados
# LIN = Regresión lineal
# TREE = Modelo de Árboles de decisión
# LIN = Regresión lineal

res2 = pd.DataFrame([['-', np.round(r2_train_lin, 2), np.round(r2_test_lin, 2)],
                    [max_depth, np.round(r_train_tree, 2), np.round(max_r2_test_tree, 2)], 
                    [n, np.round(r_train_knn, 2), np.round(max_r2_test_knn, 2)]], 
                   columns=['Parámetro', 'R2_train', 'R2_test'], index=['LIN', 'TREE', 'KNN'])
res2 = res2.transpose()
res2

In [ ]:
# Resultados del sprint 1
res1 = pd.DataFrame([['-', 0.55, 0.55],[17, 0.95, 0.77],[3, 0.88, 0.75]], 
                   columns=['Parámetro', 'R2_train', 'R2_test'], index=['LIN', 'TREE', 'KNN'])
res1 = res1.transpose()
res1

### **Modelos Avanzados**
***

#### **Elección, Entrenamiento y Evaluación de Modelos Avanzados**

##### RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestRegressor

2. Investigar sus parámetros. En particular, `n_estimators`, `max_features` y `oob_score`. Luego, crear y entrenar un modelo en el conjunto de train.

In [ ]:
clf = RandomForestRegressor(n_estimators=100, max_features='sqrt', n_jobs=-1, oob_score = True, random_state = 42)
clf.fit(X_train,y_train)


In [ ]:
import sklearn.metrics as metrics

In [ ]:
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)
print(metrics.mean_squared_error(y_train, y_train_pred, squared=False))
print(metrics.mean_squared_error(y_test, y_test_pred, squared=False))

In [ ]:
clf.oob_score_

In [ ]:
clf.feature_importances_

In [ ]:
importances = clf.feature_importances_
columns = X_train.columns
indices = np.argsort(importances)[::-1]

plt.figure(figsize = (15,8))
sns.barplot(columns[indices], importances[indices])
plt.show()

#### **Optimización de hiperparámetros**

##### GPMINIMIZE

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
# Creamos el objeto regresor
regressor = DecisionTreeRegressor()

In [ ]:
# Lo fitteamos a la data de train
regressor.fit(X_train, y_train)

In [ ]:
# Definimos aquellos parámetros que queremos colocar, pero que queremos NO mover luego
FIXED_PARAMS = {         
                'random_state': 0,           
                     }

Ahora definiremos el espacio de búsqueda. Es decir, entre que valores vamos a buscar los hiperparámetros

In [ ]:
! pip install scikit-optimize

In [ ]:
from skopt.space import Real, Integer, Categorical
cant_columnas = X_train.shape[1]
space= [          
             Categorical(['mse','mae'], name='criterion')
            ,Integer(2, 100, name='max_depth') # maxima profundidad de cada árbol (aquí sí es mejor que sean profundos, porque no se concatenan, son independientes)
            ,Integer(10, 100, name='min_samples_split') # mínima cantidad de registros para abrir una rama
            ,Integer(5, 50, name='min_samples_leaf') # mínima cantidad de registros para abrir una hoja
            ,Integer(round(cant_columnas*0.1), round(cant_columnas*0.8), name='max_features') # máxima cantidad de atributos (columnas) que puede usar cada árbol
            ]
# listamos los nombres de los parámetros cuyo espacio de búsqueda acabamos de definir
param_names = ['criterion','max_depth','min_samples_split','min_samples_leaf','max_features']

Definimos una funcion de métricas de evaluacion (para obtener RMSE)

In [ ]:
from sklearn.metrics import roc_auc_score, mean_squared_error
def eval_metrics(y_true, y_pred):
    rmse = mean_squared_error(y_true, y_pred, squared=False) 
    return rmse

Lo más importante: definimos todo lo que es nuestra "función objetivo". Es decir, lo que queremos que se haga en cada iteración!

In [ ]:
# Definimos la funcion objetivo, que se utilizará a cada iteración de la búsqueda
# creamos una lista para guardar los resultados
lista_results = []
lista_test_scores_cv = []
lista_train_scores_cv = []
lista_test_std_cv = []

from skopt.utils import use_named_args
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer

@use_named_args(space)
def objective(**params):

    # seteamos los parámetros fijos
    regressor.set_params(**FIXED_PARAMS)
    # y los parámetros de cada iteración (se setearán automáticamente así)
    regressor.set_params(**params)
    
    ####################################################################################
    ############################# USAMOS CROSS VALIDATION ##############################
    ####################################################################################
    # Ahora en vez de eso calculamos nuestros scores de test de un cross validation
    # Recordemos, este objeto devuelve un array de todos los test_scores y tambien de los train_scores hallados --> nuestra loss será el promedio de los test_scores
    model_i_scores = cross_validate(regressor, X_train, y_train, cv=5, scoring=  make_scorer(eval_metrics), return_train_score=True)
    mean_test_scores = model_i_scores['test_score'].mean()
    # Peeero tambien nos guardamos los promedios de train para poder ver si en general se estuvo overfiteando o no 
    mean_train_scores = model_i_scores['train_score'].mean()
    # Por último, resulta interesante ver cuan volatil fueron esos test_scores. Así que tomamos su desvio standard (std)
    std_test_scores = model_i_scores['test_score'].std()
    
    # Ahora guardamos estos datos en las listas vacias que creamos antes, así nos quedan bien guardados
    lista_test_scores_cv.append(mean_test_scores) # test
    lista_train_scores_cv.append(mean_train_scores) # train
    lista_test_std_cv.append(std_test_scores) # std de test
 
    # Definimos que la funcion de perdida sea el promedio de los scores de test
    loss = mean_test_scores
    ####################################################################################
    ####################################################################################
    ####################################################################################
    
    return loss 

In [ ]:
# Ejecutamos la búsqueda de los hiperparámetros 

from skopt import gp_minimize, forest_minimize, dump
# import joblib
from tqdm.notebook import tqdm

# Armamos una clase con tqdm para poder ver el progreso de la búsqueda
class tqdm_skopt(object):
    def __init__(self, **kwargs):
        self._bar = tqdm(**kwargs)
    def __call__(self, res):
        self._bar.update()

# Cantidad de iteraciones para la búsqueda (utilizar muchas, aqui usamos 50 solo por probar)
cantidad_iteraciones = 200

# Búsqueda
res = gp_minimize(
                    objective
                    ,space
                    ,n_calls = cantidad_iteraciones
                    ,n_initial_points = int(round(cantidad_iteraciones*0.2)) # Cantidad de iteraciones iniciales random (20% es significativo, pero dando mucho espacio para que el algoritmo optimice)
                    ,n_jobs=1
                    ,random_state = 0
                    ,verbose=1
                    ,callback=[tqdm_skopt(total=cantidad_iteraciones, desc="Gaussian Process")]
                    )

In [ ]:
res.x

In [ ]:
res.fun

In [ ]:
res.space

In [ ]:
res.specs

In [ ]:
res.x_iters

In [ ]:
res.func_vals 

Armamos un dataframe con los resultados

In [ ]:
df_vemos_que_paso = pd.DataFrame(res.x_iters)
df_vemos_que_paso.columns = param_names
df_vemos_que_paso['funcion_costo'] = res.func_vals
df_vemos_que_paso['numero_de_iteracion'] = df_vemos_que_paso.reset_index()['index']
df_vemos_que_paso['score_train'] = lista_train_scores_cv[:200]
df_vemos_que_paso['score_test'] = lista_test_scores_cv[:200]
df_vemos_que_paso['dif_train_test'] = df_vemos_que_paso['score_test'] - df_vemos_que_paso['score_train']
df_vemos_que_paso['std_test'] = lista_test_std_cv[:200]
df_vemos_que_paso.sort_values('funcion_costo', ascending=True, inplace=True)
df_vemos_que_paso

In [ ]:
import matplotlib.pyplot as plt
plt.plot(df_vemos_que_paso['numero_de_iteracion'], df_vemos_que_paso['funcion_costo'], '--')

plt.legend()
plt.show()

Armamos un diccionario con los mejores hiperparametros

In [ ]:
# Armamos un diccionario a partir de 2 listas: la primera es los nombres que tienen los hiperparametros, y la otra son los mejores que se encontraron
mejores_hp = dict(zip(param_names, res.x))
# Pero adem{as le agregamos aquellos hiperparametros que dejamos fijos (en nuestro caso solo fue el random_state)
mejores_hp.update(FIXED_PARAMS)

In [ ]:
mejores_hp

In [ ]:
# Colocamos esos mejores hiperparametros en nuestro modelo
regressor.set_params(**mejores_hp)

In [ ]:
# Entrenamos ese arbol que ya tiene los hiperparametros correctos
regressor.fit(X_train,y_train)

In [ ]:
pip install shap

In [ ]:
import shap

In [ ]:
# Utilizamos el explicador de SHAP
explainer = shap.TreeExplainer(regressor)
# Y pedimos que nos otorgue los valores de importancia que tendrá en el dataframe de validacion
shap_values = explainer.shap_values(X_test)

In [ ]:
# Graficamos solo la importancia de cada variable
shap.summary_plot(shap_values, X_test, plot_type="bar")

In [ ]:
# Graficamos el impacto de cada variable 
# recordemos, para leer el grafico: 
# - los colores significan los valores de la variable --> ej: un punto de V5 que esté en color fucsia, es un valor alto de V5
# - el eje x significa cuanto impacta sobre el output --> ej: los puntos fucsias de V5 están a la derecha, así que impactan positivamente sobre el output
# Entonces: A mayor valor de V5, mayor valor del output en general. Y así se podría analizar cada variable

shap.summary_plot(shap_values, X_test)

In [ ]:
# Graficamos como queda el arbol terminado
from sklearn import tree
import matplotlib.pyplot as plt
plt.figure(figsize=(20,20))
tree.plot_tree(regressor) 


##### OPTUNA

In [ ]:
! pip install optuna

In [ ]:
param_names

In [ ]:
import joblib

In [ ]:
def objective(trial):    
    
    joblib.dump(study, 'study.pkl')
    
    tree_criterion = trial.suggest_categorical('criterion', ['mse', 'mae']) 
    tree_max_depth = trial.suggest_int('max_depth', 2, 200) 
    tree_min_samples_split = trial.suggest_int('min_samples_split', 10, 100) 
    tree_min_samples_leaf = trial.suggest_int('min_samples_leaf', 5, 50) 
    tree_max_features = trial.suggest_int('max_features', round(cant_columnas*0.1), round(cant_columnas*0.8)) 

    params = {
        'criterion': tree_criterion,
        'max_depth': tree_max_depth,
        'min_samples_split': tree_min_samples_split,
        'min_samples_leaf': tree_min_samples_leaf,
        'max_features': tree_max_features
    }
    
    regressor.set_params(**params)

    ####################################################################################
    ############################# USAMOS CROSS VALIDATION ##############################
    ####################################################################################
    # Ahora en vez de eso calculamos nuestros scores de test de un cross validation
    # Recordemos, este objeto devuelve un array de todos los test_scores y tambien de los train_scores hallados --> nuestra loss será el promedio de los test_scores
    model_i_scores = cross_validate(regressor, X_train, y_train, cv=3, scoring=  make_scorer(eval_metrics), return_train_score=True)
    mean_test_scores = model_i_scores['test_score'].mean()
    # Peeero tambien nos guardamos los promedios de train para poder ver si en general se estuvo overfitteando o no 
    mean_train_scores = model_i_scores['train_score'].mean()
    # Por último, resulta interesante ver cuan volatil fueron esos test_scores. Así que tomamos su desvio standard (std)
    std_test_scores = model_i_scores['test_score'].std()
    
    # Ahora guardamos estos datos en las listas vacias que creamos antes, así nos quedan bien guardados
    lista_test_scores_cv.append(mean_test_scores) # test
    lista_train_scores_cv.append(mean_train_scores) # train
    lista_test_std_cv.append(std_test_scores) # std de test
 
    # Definimos que la funcion de perdida sea el promedio de los scores de test
    loss = mean_test_scores
    ####################################################################################
    ####################################################################################
    ####################################################################################

    return loss

In [ ]:
import optuna

In [ ]:
study = optuna.create_study()


In [ ]:
study.optimize(objective, n_trials=200)

In [ ]:
study.best_trial

In [ ]:
study.best_value

#### **Comparación de Modelos**

##### **Comparación de los tres modelos**

###### A quien le fue mejor?

In [ ]:
print(f"El RMSE al que llegó GP_MINIMIZE fue {res.fun}")
print(f"El RMSE al que llegó OPTUNA fue {study.best_value}")

In [ ]:
if float(res.fun) > float(study.best_value):
    print("El ganador fue OPTUNA porque llegó al menor valor")
    
elif float(res.fun) < float(study.best_value):
    print("El ganador fue GP_MINIMIZE porque llegó al menor valor")
    
elif float(res.fun) == float(study.best_value):
    print("Empate")

### **Interpretación de Modelos**
***

#### **Responder preguntas planteadas**

#### **Distribución de los errores**

#### **Fallas de los modelos**

### **Cierre**
***

#### **Conclusiones**